In [2]:
import sqlite3 as sql3
import pandas as pd
#from reverso_context_api import Client

connectionToDataBase = sql3.connect("dictionary.db")

#mCursor = connectionToDataBase.cursor()

#mTableSelectedTable = mCursor.execute("SELECT * FROM searched")

""" for mRow in mTableSelectedTable:
    display(mRow) """

df_searched = pd.read_sql_query("SELECT* FROM searched",connectionToDataBase)

connectionToDataBase.close()

display(df_searched)

,seen_column,searched_column,lang,_id
0,... arıyorum.,... ariyorum.,tur.,1
1,... bilet lütfen,... bilet lutfen,tur.,2
2,... görmek istiyorum.,... gormek istiyorum.,tur.,3
3,... için hangi hattı kullanmam lazım,... icin hangi hatti kullanmam lazim,tur.,4
4,... istiyorum.,... istiyorum.,tur.,5
...,...,...,...,...
167268,Zürih,Zurih,tur.,167269
167269,zygoma,zygoma,eng.,167270
167270,zymosis,zymosis,eng.,167271
167271,zymotic,zymotic,eng.,167272


In [3]:
series_seen_column = df_searched[["seen_column","lang"]]
display(series_seen_column)

,seen_column,lang
0,... arıyorum.,tur.
1,... bilet lütfen,tur.
2,... görmek istiyorum.,tur.
3,... için hangi hattı kullanmam lazım,tur.
4,... istiyorum.,tur.
...,...,...
167268,Zürih,tur.
167269,zygoma,eng.
167270,zymosis,eng.
167271,zymotic,eng.


In [4]:
from itertools import islice
from reverso_context_api import Client
import json

# The following add timout functionality to my code in case the API delays/doesn't return examples/output
import time
from call_function_with_timeout import SetTimeout
from call_function_with_timeout import SetTimeoutDecorator
import random

num = 5 #Number of pairs of examples per word
current_row = 0

example_dictionary = {} # creating an empty dictionary
example_dictionary_lang_filter = {}

for word in series_seen_column["seen_column"]:
#for word,lang in [("merhaba","tur."),("tall","eng."),("günaydın","tur."),("short","eng.")]:

    lang = series_seen_column.loc[current_row,"lang"]

    if lang =="tur.":
        source_lang="tr"
        target_lang="en"
    else:
        source_lang="en"
        target_lang="tr"

    current_row = current_row + 1

    @SetTimeoutDecorator(timeout = 120) # This decorator jumps the function when timout is reached
    def get_samples(client, word, num):
        # There can be thousands of translation samples, this function requests and returns only needed amount of them
        iterator = client.get_translation_samples(word)
        return list(islice(iterator, num))

    client = Client(source_lang, target_lang)
    # call get_samples for each word in your list
    is_done, is_timeout, erro_message, example_list = get_samples(client, word,num)

    example_dictionary[word] = example_list # Here I am appending the list of pairs of examples to each searched word. Where word is the key...
    #example_dictionary_lang_filter[source_lang] = example_dictionary
    example_dictionary_lang_filter = example_dictionary

    with open('examples_db.json','w',encoding='utf-8') as f:
     json.dump(example_dictionary,f,ensure_ascii=False)
    print("========================PROCESS INFO=============================")
    print("Row #: ",current_row)
    print("Progress: ", round((current_row/167273)*100,3), " %")
    wait_time = random.choice([15,20,10,18,23,16,11,13,16,19,6,22,8])
    print("Time for next request: ",wait_time," seconds")

    print("==========================OUTPUT INFO===========================")
    print("Current Word: ", word)
    print(example_list)

    time.sleep(wait_time)

#with open('examples_db.json','w',encoding='utf-8') as f:
    #json.dump(example_dictionary_lang_filter,f,ensure_ascii=False)

========================PROCESS INFO=============================
Row #:  1
Progress:  0.001  %
Time for next request:  15  seconds
==========================OUTPUT INFO===========================
Current Word:  ... arıyorum.
[('Bir kayıp çocuk için... her evi arıyorum.', 'Searching every house... for a missing child.'), ('Dr. McNamara... nişan törenimden arıyorum sizi.', "Dr. McNamara... I'm calling you from my engagement party."), ('Ben hâlâ bundan daha olağanüstü... bir şey arıyorum.', "I'm still looking for something... more extraordinary than that."), ('Ben de bir çocuğu arıyorum... kendi çocuğumu.', "I'm looking for a kid... my kid."), ('Bir şey arıyorum... birazcık farklı.', 'I am seeking for something... a little different.')]
========================PROCESS INFO=============================
Row #:  2
Progress:  0.001  %
Time for next request:  23  seconds
==========================OUTPUT INFO===========================
Current Word:  ... bilet lütfen
[("Dediğim gibi, Paris'e t